# K-MCI
### Intro
In this notebook we have the main algorithm of the article. All necessary functions are implemented in function.py file.<br>
This document have blow sections:<br>
1. Importing Libraries & classes
2. Reading datasets
3. Preprocessing datasets
4. Main Algorithm

### Importing Libraries & classes
We import some libraries from third party.

In [21]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import random
import copy
from sklearn import preprocessing

In [22]:
# defnition of Candidate class
class Candidate:    
    def __init__(self,nodes,sampling_interval,fitness=-1):
        self.nodes = nodes
        self.sampling_interval = sampling_interval 
        self.variation = []
        self.random_number
        self.fitness = fitness
        self.centers = []
        self.probability = []
        self.labels = []
        self.each_fitness = []
        
    def random_number(self):
        self.random_number = random.random()
        
    def describe(self):
        print("Candidate with #{} nodes, #{} centers, fitness={}\n clusters:\n{} \n Centers:\n{}\n sampling_interval:\n{}"
              .format(len(self.nodes),len(self.centers),self.fitness,self.nodes[np.random.randint(self.nodes.shape[0], size=5), :],self.centers,self.sampling_interval))

### Reading Datasets and Preprocessing
We show the structure of our data for all of datasets.<br>
We have these datasets from UCI Machine Learning Repository:
1. Breast Cancer Wisconsin
2. Contraceptive Method Choice data
3. Glass
4. Iris
5. Vowel
6. Wine

In [23]:
""" In this cell we have some code which will do preprocessing for us on all datasets """
""" the resourses of these datasets are mentioned in README.md file. """

path = 'datasets/' 
# because we are using local files, you need to download datasets and change the "path" variable
# local folder of your downloaded datasets


def bcw(): 
    # importing dataset
    dataset = pd.read_csv(path+'bcw.csv',
                          names = ['Sample code number','Clump Thickness','Uniformity of Cell Size',
                                   'Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial',
                                   'Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses','Class'])
    dataset = dataset.drop(dataset.columns[0], axis=1)
    dataset= dataset.replace(to_replace='?',value=np.nan)
    dataset = dataset.dropna(axis =0) # resolving missing values
    dataset = dataset.astype('int64') # casting string valued column to int64
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def cmc():
    # importing dataset
    dataset = pd.read_csv(path+'cmc.csv', names=["Wife's age","Wife's education","Husband's education",
                                                 "Number of children ever born","Wife's religion",
                                                 "Wife's now working?","Husband's occupation",
                                                 "Standard-of-living index","Media exposure","Contraceptive method used"])
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def glass():
    # importing dataset
    dataset = pd.read_csv(path+'glass.csv', names= ['Id','refractive index','Sodium','Magnesium','Aluminum','Silicon','Potassium'
                                                    ,'Calcium','Barium','Iron','glass'])
    x = dataset.iloc[:,1:10].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def iris():
    # importing dataset
    dataset = pd.read_csv(path+'iris.csv', names= ['sepal length','sepal width','petal length','petal width','class'])
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    
    # encoding categorial data types to labelEncoder
    from sklearn.preprocessing import LabelEncoder
    labelencoder_y = LabelEncoder()
    labelencoder_y = labelencoder_y.fit(y)
    y = labelencoder_y.transform(y)  # 0 for 'Iris-setosa', 1 for 'Iris-versicolor', 2 for 'Iris-virginica'
    return (x,y,dataset)


def vowel():
    # importing dataset
    dataset = pd.read_csv(path+'vowel.csv', names= ['vowel','type 1 frq','type 2 frq','type 3 frq'])
    x = dataset.iloc[:,1:].values # features
    y = dataset.iloc[:,0].values # target values
    return (x,y,dataset)

def wine():
    # importing dataset
    dataset = pd.read_csv(path+'wine.csv', names= ['class','Alcohol','Malic acid','Ash','Alcalinity of ash','Magnesium','Total phenols','Flavanoids','Nonflavanoid phenols','Proanthocyanins','Color intensity','Hue','OD280/OD315','Proline'])
    x = dataset.iloc[:,1:].values # features
    y = dataset.iloc[:,0].values # target values
    return (x,y,dataset)

In [24]:
# Getting data from our preprocessing class Datasets
x_bcw, y_bcw, df_bcw = bcw()
x_cmc, y_cmc, df_cmc = cmc()
x_glass, y_glass, df_glass = glass()
x_iris, y_iris, df_iris = iris()
x_vowel, y_vowel, df_vowel = vowel()
x_wine, y_wine, df_wine = wine()

In [25]:
# Showing dataset of Breast Cancer Wisconsin
df_bcw.head(3)

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2


In [26]:
# Showing dataset of Contraceptive Method Choice data
df_cmc.head(3)

,Wife's age,Wife's education,Husband's education,Number of children ever born,Wife's religion,Wife's now working?,Husband's occupation,Standard-of-living index,Media exposure,Contraceptive method used
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1


In [27]:
# Showing dataset of Glass
df_glass.head(3)

,Id,refractive index,Sodium,Magnesium,Aluminum,Silicon,Potassium,Calcium,Barium,Iron,glass
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1


In [28]:
# Showing dataset of Iris
df_iris.head(3)

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


In [29]:
# Showing dataset of Vowel
df_vowel.head(3)

,vowel,type 1 frq,type 2 frq,type 3 frq
0,1,700,1500,2600
1,1,550,1550,2400
2,1,700,1500,2600


In [30]:
# Showing dataset of Wine
df_wine.head(3)

,class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185


### Standardization
In this cell, we standard our data using this equation: <img src='http://3.bp.blogspot.com/_xqXlcaQiGRk/RpO4yR0oKtI/AAAAAAAAABM/7rUWCMwizus/s200/fig2.png'></img>

In [31]:
x_bcw = preprocessing.scale(x_bcw)
x_wine = preprocessing.scale(x_wine)
x_cmc = preprocessing.scale(x_cmc)
x_glass = preprocessing.scale(x_glass)
x_iris = preprocessing.scale(x_iris)
x_vowel = preprocessing.scale(x_vowel)

C:\Users\HaMeD\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


# Main Algorithm 

The implementation of main algorithm have below sections:
1. <Strong>Initializing Parameters and Candidates</Strong>
    1. Sampling Interval
    2. Apply Random Centers to Candidates
    3. Visualizing Candidates
2. <Strong>Running Kmean to Exploit</Strong>
    1. K-means Algorithm
    2. Visualizing Candidates
3. <Strong>Mutation Algorithm</Strong>
    1. Fitness over Mutated Candidate
    2. Probablility Equation
    3. Mutation Logic

**Extra details will be explain in other next sections**


## Initializing Parameters and Candidates
In this step, we initialize our algorithm. There are some parameters which are important to convergance speed and quality of solution. So we initialize them here based on the paper.<br>
Each time you want to run K-MCI algorithm, you should first do this initialization.<br>

In [32]:
# notice: this values related to IRIS dataset
candidates_array = [] # the array of all candidates
number_of_candidates = 5
sampling_interval_reduction_factor = 0.95
convergence_parameter = None # what is this in paper???
mutation_random = 0.7
iterations_count = 3500
variations_count = 15
number_of_clusters = 3 # in this article number of clusters are predefined

## Sampling Interval
this function use for making sampling_interval


In [33]:
def sampling_intervals(input_array):
    Maxes = np.amax(input_array,axis=0)
    Mins  = np.amin(input_array,axis=0)
    return (Maxes,Mins)

In [34]:
# initializing candidates
for i in range(0 ,number_of_candidates):
    candidates_array.append(Candidate(x_wine,sampling_interval = sampling_intervals(x_wine)))

## Visualizing Candidates
some parameters are uninitialized. They will have value after running Kmean and Cohort and mutations.

In [35]:
candidates_array[random.randint(0,number_of_candidates-1)].describe()

Candidate with #178 nodes, #0 centers, fitness=-1
 clusters:
[[ 1.0615645  -0.88540853 -0.352802   -1.04947918 -0.12228166  1.09741707
   1.12517596 -1.14303148  0.45396697  0.93517742  0.23055711  1.32531572
   0.94931905]
 [ 0.35746347 -0.32885738  1.14588777 -0.80925118  0.1585717   1.1294637
   1.2054933  -0.41782893  0.12107098  0.40744204  0.49379744  0.32248133
   1.66583014]
 [-0.81603824  0.10202093  0.34171277  0.45194578 -0.12228166  0.42443782
   0.08105057 -0.17609475 -0.49215846 -0.9767819  -0.69078402  1.08520045
  -0.98366865]
 [ 1.70390229 -0.4186237   0.0492855  -2.25061915  0.1585717   1.61016317
   1.61711966 -0.57898505  2.39878035  1.05629702  1.06415147  0.54847218
   2.54793491]
 [ 0.44393202  0.20076388 -0.06037473  0.15166079 -0.75420173 -1.43426677
  -1.53533585  0.06563943 -1.66605483  0.23441405 -1.12951789 -0.20012252
   0.10542821]] 
 Centers:
[]
 sampling_interval:
(array([2.25977152, 3.10919247, 3.15632527, 3.15451071, 4.37137214,
       2.53951547, 3.0

## Apply Random Centers to Candidates
In this step, we just select some random nodes as centers with psuedorandom functions in python native library.

In [36]:
for c in range(0,number_of_candidates):
    candidates_array[c].centers =  np.asarray(random.sample(list(candidates_array[c].nodes),number_of_clusters))
    print(candidates_array[c].centers)

[[-0.6060432  -0.54429654 -1.41285087  0.30180329 -1.03505509 -0.15240153
  -0.10970311 -0.33725087 -0.19430416 -0.9162221   0.36217728  1.35356458
  -0.23849711]
 [-0.97662269 -1.02903463 -2.25357928 -0.80925118  3.59902539 -0.71321758
  -0.75224181 -1.78765596  1.59282165 -0.9551534   1.41513857  0.64734317
  -0.0920104 ]
 [-1.18661773  1.76269775  0.0492855   0.75223078 -1.38612179 -0.31263469
  -0.28037745  0.46852973 -0.4220751  -1.0632959  -0.7346574  -0.05887823
  -0.53147054]]
[[ 0.24628963 -0.49941338 -0.82799632 -2.49084714  0.01814502  0.56864766
   0.73362894 -0.82071924 -0.54472099 -0.29332133  0.40605066  1.1134493
   0.96524152]
 [ 1.3086175  -0.16727801  0.89001391 -0.56902319  1.49262517  0.48853108
   0.48263726 -0.41782893 -0.59728351 -0.00349944  0.44992405  1.36768901
   1.74544249]
 [ 0.71569031  0.21871714  1.18244118  1.50294326  0.36921172 -1.19391704
  -1.19398717  0.22679555 -0.08917911  1.5580782  -0.95402434 -1.1464592
   0.0098934 ]]
[[-0.6060432  -0.54429

## K-means Algorithm 
this function fitting k-means model on dataset

In [37]:
def doKmeans(x,clusters_count,init_centers):
    kmeans = KMeans(n_clusters = clusters_count, init = init_centers,n_init = 50)
    kmeans = kmeans.fit(x)
    labels = kmeans.labels_
    return (labels,kmeans.cluster_centers_,kmeans.inertia_)


## Running Kmean to Exploit
Here by running Kmean on all of our candidates, they centers initialized and nodes assigns to clusters based on Kmean++ algorithm.<br>
The objective function of Kmean is also the object function of K-MCI. <br>
So the fitness of any candidates is calculated using Kmean inertia or mean square error of nodes to their centers distances.

In [38]:

for c in range(0,number_of_candidates):
    y_predict_temp,centers_temp,fitness_temp = doKmeans(candidates_array[c].nodes,clusters_count = number_of_clusters,init_centers = candidates_array[c].centers)
    candidates_array[c].centers = centers_temp
    candidates_array[c].fitness = fitness_temp
    candidates_array[c].labels   = y_predict_temp
    
candidates_array[2].labels

C:\Users\HaMeD\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=50
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 2, 2, 2, 2,
       2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

## Visualizing Candidates
In this step, by Kmeans, we assigned centers to each candidates and calculated the fitness of each candidate using inertia

In [39]:
candidates_array[random.randint(0,number_of_candidates-1)].describe()

Candidate with #178 nodes, #3 centers, fitness=1586.1901646133865
 clusters:
[[-9.27212090e-01 -5.44296535e-01 -9.01103141e-01 -1.48624201e-01
  -1.38612179e+00 -1.03368389e+00  7.33234123e-04  6.56394314e-02
   6.85084581e-02 -7.17239912e-01  1.86683727e-01  7.88587455e-01
  -7.54385098e-01]
 [-1.48995162e-01  5.86759028e-01  1.22392314e-01  1.51660791e-01
   2.98998383e-01 -1.59449993e+00 -8.12479811e-01 -9.81875357e-01
  -1.33315885e+00  1.47900054e-01 -9.54024339e-01 -1.68318747e+00
  -6.90695223e-01]
 [-7.78980294e-01 -1.08289442e+00 -7.54889505e-01 -1.48624201e-01
  -8.94628408e-01  1.93062948e+00  1.07497763e+00 -1.38476566e+00
   4.89008649e-01 -2.63041430e-01  1.15189824e+00  3.64854610e-01
  -1.04098953e+00]
 [-6.06043199e-01 -9.84151477e-01 -4.25908823e-01 -5.99051690e-01
  -1.03505509e+00 -4.72867844e-01 -1.45501851e+00  1.91893483e+00
  -5.97283511e-01  1.69528553e-01 -9.10150952e-01 -1.55606762e+00
  -3.08555975e-01]
 [-1.13720713e+00 -9.03361794e-01 -2.43141777e-01  1.23

In [42]:
sum_of_squares(candidates_array[0].nodes,candidates_array[0].centers,candidates_array[0].labels)

1586.1901646133865

## Fitness over Mutated Candidate
Just simple sum of square of distances

In [41]:
def sum_of_squares(data, centroids, labels):
    sqe = 0
    for l in np.unique(labels):
        data_l = data[labels == l]
        resid = data_l - centroids[l]
        sqe += (resid**2).sum()
    return sqe

### Mean Square Distance to a Specific Center
This function is exactly based on "sum_of_squares" function. But here all centers not engaged because we want to find the sum of square distances of nodes of a cluster to their cluster center.<br>
The idea is when mutation process done, at least one dimension may get better center but the other goes bad. So we want to have that center which is better to the previous time.

In [43]:
def center_dist(data, center, labels, center_position, center_cluster):
    sqe = 0
    for l in range(0,len(labels)):
        if center_cluster == labels[l] :
            data_l = data[l][center_position]
            resid = data_l - center
            sqe += (resid**2)
    return sqe

In [44]:
def each_fitness(candidates_array_example):
    fitness_of_each_centers = []
    for i in range(0,len(candidates_array_example.centers)):
        Second_layer = []
        for j in range(0,len(candidates_array_example.centers[i])):
            Second_layer.append(center_dist(candidates_array_example.nodes,candidates_array_example.centers[i][j],candidates_array_example.labels,j,i))
        fitness_of_each_centers.append(Second_layer)
    return np.array(fitness_of_each_centers)
print(each_fitness(candidates_array[0]))

[[1.30322470e+02 5.87297640e+01 1.26255005e+02 1.08604675e+02
  9.16792222e+01 5.90643551e+01 4.64490943e+01 4.37291687e+01
  6.97389728e+01 5.59886415e+01 5.79288009e+01 3.31931412e+01
  1.44548571e+02]
 [1.04663129e+00 3.95637700e+00 8.58717986e+00 2.40468227e+00
  8.50903019e+00 4.16627906e+00 1.77209561e+00 2.49843862e+00
  7.51339396e+00 9.16872272e-02 3.16834277e+00 9.26315942e-01
  8.31249688e-01]
 [3.84409432e+01 7.01013451e+01 3.52830389e+01 3.96468351e+01
  4.00710930e+01 2.11239285e+01 1.39407652e+01 5.84859588e+01
  3.49358405e+01 8.95182814e+01 3.71283884e+01 2.48640487e+01
  1.09461125e+01]]


## Mutation algorithm 
this function do mutation algorithm on candidates interval's centers

In [46]:
def mutation(candidate_array,mutation_random):
    New_candidate = copy.deepcopy(candidate_array)
    Mutant_candidate = copy.deepcopy(candidate_array)
    Trial_candidate = copy.deepcopy(candidate_array)
    for x in range(len(candidate_array)):
        a = np.full(len(candidate_array), 1/(len(candidate_array)-1))
        a[x] = 0
        temp = np.random.choice(len(candidate_array), 3, replace = False, p=a)
        for i in range(len(candidate_array[0].centers)):
            for j in range(len(candidate_array[0].centers[i])):
                Mutant_candidate[x].centers[i][j] = candidate_array[temp[0]].centers[i][j]+ random.random()*(candidate_array[temp[1]].centers[i][j] - candidate_array[temp[2]].centers[i][j])
                if random.random() < mutation_random:
                    Trial_candidate[x].centers[i][j] = Mutant_candidate[x].centers[i][j] #Trial & Mutant must be copy of same candidate
                    
                    
        Trial_candidate[x].fitness = sum_of_squares(Trial_candidate[x].nodes,Trial_candidate[x].centers,Trial_candidate[x].labels)
        
        if Trial_candidate[x].fitness < sum_of_squares(candidate_array[x].nodes,candidates_array[2].centers,candidates_array[2].labels):
            New_candidate[x] = Trial_candidate[x]
            print("sakjfnr")
        else:
            New_candidate[x] = candidate_array[x]
        #print(Trial_candidate[x].centers)
        #print(candidate_array[x].centers)
        #print(New_candidate[x].centers)
        #print(Mutant_candidate[x].centers)
        print("###########################################################################################################")
        Trial_candidate[x].each_fitness = each_fitness(Trial_candidate[x])
        print(Trial_candidate[x].each_fitness)
        candidate_array[x].each_fitness = each_fitness(candidate_array[x])
        print(candidate_array[x].each_fitness)
        print("###########################################################################################################")
        
    return New_candidate
#print(candidates_array[1].fitness)
#for i in range(0,3500):
New_candidates_array = mutation(candidates_array,mutation_random)
    #if New_candidates_array[1].fitness != candidates_array[1].fitness:
     #   print(New_candidates_array[1].fitness)#never happen

###########################################################################################################
[[130.3224704   59.02041453 126.25500478 124.53448259  91.67922216
   65.96244528  53.63968225  45.80903658  78.3870868   74.87711949
   57.92880089  34.33725039 221.35733963]
 [  9.91456753   3.956377    15.67185757   2.40468227   8.50903019
    7.66529785   1.77209561  17.3602583   46.32292822  18.0788926
   14.82646746   9.35690201   0.83124969]
 [ 87.31931735  70.10134513  57.43650669  39.64683513  51.51483914
   74.918207    96.968094   103.15297789  34.93584051 221.8755219
  150.39605007 145.91229001  17.81965463]]
[[1.30322470e+02 5.87297640e+01 1.26255005e+02 1.08604675e+02
  9.16792222e+01 5.90643551e+01 4.64490943e+01 4.37291687e+01
  6.97389728e+01 5.59886415e+01 5.79288009e+01 3.31931412e+01
  1.44548571e+02]
 [1.04663129e+00 3.95637700e+00 8.58717986e+00 2.40468227e+00
  8.50903019e+00 4.16627906e+00 1.77209561e+00 2.49843862e+00
  7.51339396e+00 9.16872272e-02 3.168

# Probability Equation

In [47]:
def probability(candidates):
    for c in range(0,len(candidates)):
        temp_array = []
        for clusters in range(0,len(candidates[c].centers)):
            temp_array.append((1/np.sum(candidates[c].each_fitness[clusters]))/(np.sum(1/(np.sum(x.each_fitness[clusters])) for x in candidates)))
        print(temp_array)    
        candidates[c].probability = temp_array
    return

### Roulette Wheel Selection
This is a logic for selecting targets with higher probability.<br>
In this article, each candidate want to follow the other candidate with higher fitness. So in the previous section <strong><em>Probability Equation<strong><em>, we calculated all probabilities of candidates based on their fitnesses.

In [48]:
def roulette_wheel_selection(inertia_array):
    maximum = np.sum(inertia_array)
    pick = random.uniform(0, maximum)
    current = 0
    for fitness in inertia_array:
        current += fitness
        if current > pick:
            return fitness

In [49]:
def shrinked_sampling_interval(candidates_array):
    for i in range(0,len(candidates_array)):
        shrinked_sampling_interval = []
        for j in range(0,len(candidates_array[i].centers)):
            Second_layer = []
            for k in range(0,len(candidates_array[i].centers[j])):
                Third_layer = []
                Temp = abs(candidates_array[i].sampling_interval[0][k] - candidates_array[i].sampling_interval[1][k]) * sampling_interval_reduction_factor
                Third_layer.append(candidates_array[i].centers[j][k] - (Temp/2))
                Third_layer.append(candidates_array[i].centers[j][k] + (Temp/2))
                Second_layer.append(Third_layer)
            shrinked_sampling_interval.append(Second_layer)
        candidates_array[i].sampling_interval = shrinked_sampling_interval
    return
shrinked_sampling_interval(New_candidates_array)
print(New_candidates_array[0].sampling_interval)

[[[-2.1317098070406173, 2.327596716693815], [-2.539834954859721, 1.7752317860201188], [-3.3146773611370715, 3.179035767689012], [-3.061722276436061, 2.472530131453211], [-3.099587352686193, 3.037058618210886], [-1.6326294840855602, 2.781793931443605], [-1.61957609584806, 2.9012860548379478], [-2.4866706392209106, 1.5704347137895789], [-2.3000659408947377, 2.9763354172188383], [-2.6909846492732905, 2.125249541023684], [-2.0491934540883143, 3.0774118352148463], [-1.1882962813481468, 2.4748741602562423], [-1.8437650228920117, 2.3976621875573527]], [[-3.396506754838916, 1.0627997688955166], [-2.538455213358157, 1.7766115275216832], [-4.250309251418983, 2.2434038774071], [-3.1199441999410156, 2.4143082079482565], [-0.38207102096848455, 5.754574949928594], [-2.3499992528146927, 2.0644241627144724], [-2.4966339876916117, 2.024228162994396], [-3.026543645480499, 1.0305617075299902], [-0.6318871729247251, 4.644514185188851], [-3.363270497793129, 1.4529636925038454], [-2.2011253688729693, 2.9254

In [50]:
def variation(candidates_array):
    for candidate in candidates_array:
        variations = []
        for i in range(0,variations_count):
            Second_layer = []
            for j in candidate.sampling_interval:
                Third_layer = []
                for k in range(0,len(j)):
                    #print(k)
                    Third_layer.append(random.uniform(j[k][0],j[k][1]))
                Second_layer.append(Third_layer)
            variations.append(Second_layer)
        candidate.variation = variations    
variation(New_candidates_array)
print(New_candidates_array[3].variation[1])

[[-0.46903516677587254, -1.1837475039616863, 2.34278124795549, -0.8597142499404502, 2.8932116220492716, -1.992337439558609, -2.473283336818527, 1.1609910320135577, -0.7341487809951919, 2.498415217209172, -2.0307966261592947, -2.77808540607671, -1.966656781169545], [0.6621387651092507, 0.6989864324675721, -1.1724177548936363, -2.523258801741272, -2.795913548873211, -1.8295704151498788, 1.4678991593546171, -0.6563145082180608, 2.2082129621371727, 0.49143743191380107, -1.2307635874840845, -0.14877817089986856, 1.0555785952179306], [2.379687225117771, 0.25983939877249584, 0.7003250862619899, -2.587472417790643, -0.7584476932537179, 1.9996916405647445, 2.4447719909681114, 0.6957747277626285, -0.4234262653079186, -0.7690301198507956, -2.029062304222498, -0.688552751817554, 1.6263085882337884]]
